# 4.7 피봇테이블과 그룹분석

## 피봇테이블

```{margin}
`pivot`
```

피봇테이블(pivot table)이란 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용하여 데이터를 조회하여 펼쳐놓은 것을 말한다.

판다스는 피봇테이블을 만들기 위한  `pivot` 메서드를 제공한다. 첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름을 넣는다.

판다스는 지정된 두 열을 각각 행 인덱스와 열 인덱스로 바꾼 후 행 인덱스의 라벨 값이 첫번째 키의 값과 같고 열 인덱스의 라벨 값이 두번째 키의 값과 같은 데이터를 찾아서 해당 칸에 넣는다. 만약 주어진 데이터가 존재하지 않으면 해당 칸에 `NaN` 값을 넣는다.

다음 데이터는 각 도시의 연도별 인구를 나타낸 것이다.

In [1]:
import numpy as np
import pandas as pd
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


이 데이터를 도시 이름이 행 인덱스가 되고 연도가 열 인덱스가 되어 행과 열 인덱스만 보면 어떤 도시의 어떤 시점의 인구를 쉽게 알 수 있도록 피봇테이블로 만들어보자. `pivot` 명령으로 사용하고 행 인덱스 인수로는 `"도시"`, 열 인덱스 인수로는 `"연도"`, 데이터 이름으로 `"인구"`를 입력하면 된다.

In [2]:
df1.pivot("도시", "연도", "인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


이 피봇테이블의 값 3512547은 "도시"가 부산이고 "연도"가 2005년인 데이터를 "인구"열에서 찾은 값이다. 2005년 인천의 인구는 데이터에 없기 때문에 `NaN`으로 표시된다.

 피봇테이블은 다음과 같이 `set_index` 명령과 `unstack` 명령을 사용해서 만들 수도 있다.

In [3]:
df1.set_index(["도시", "연도"])

인구   지역
도시 연도                
서울 2015  9904312  수도권
   2010  9631482  수도권
   2005  9762546  수도권
부산 2015  3448737  경상권
   2010  3393191  경상권
   2005  3512547  경상권
인천 2015  2890451  수도권
   2010   263203  수도권

In [4]:
df1.set_index(["도시", "연도"])[["인구"]]

인구
도시 연도           
서울 2015  9904312
   2010  9631482
   2005  9762546
부산 2015  3448737
   2010  3393191
   2005  3512547
인천 2015  2890451
   2010   263203

In [5]:
df1.set_index(["도시", "연도"])[["인구"]].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

행 인덱스나 열 인덱스를 리스트로 주는 경우에는 다중 인덱스 피봇 테이블을 생성한다. (주의: 판다스 버전 1.1 미만에서는 버그로 인해 동작하지 않는다.)

#### 설치 후 주피터 커널 리부팅 해야 한다.

In [ ]:
!pip install --upgrade  --user pandas 

In [6]:
import pandas as pd
print(pd.__version__)
df1.pivot(["지역", "도시"], "연도", "인구")

1.3.4


연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

행 인덱스와 열 인덱스는 데이터를 찾는 키(key)의 역할을 한다. 따라서 키 값으로 데이터가 단 하나만 찾아져야 한다.
만약 행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러가 발생한다. 예를 들어 위 데이터프레임에서 ("지역", "연도")를 키로 하면 ("수도권", "2015")에 해당하는 값이 두 개 이상이므로 다음과 같이 에러가 발생한다. 

In [8]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [7]:
df1.pivot("지역", "연도", "인구")

ValueError: Index contains duplicate entries, cannot reshape

In [9]:
try:
    df1.pivot("지역", "연도", "인구")
except ValueError as e:
    print("ValueError:", e)

ValueError: Index contains duplicate entries, cannot reshape


## 그룹분석

만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석(group analysis)을 해야 한다.

그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산한다. 판다스에서는 `groupby` 메서드를 사용하여 다음처럼 그룹분석을 한다.

1. 분석하고자 하는 시리즈나 데이터프레임에 `groupby` 메서드를 호출하여 그룹화를 한다.
1. 그룹 객체에 대해 그룹연산을 수행한다.

```{margin}
`groupby`
```

### `groupby` 메서드

`groupby` 메서드는 데이터를 그룹 별로 분류하는 역할을 한다. `groupby` 메서드의 인수로는 다음과 같은 값을 사용한다.

 * 열 또는 열의 리스트 
 * 행 인덱스

연산 결과로 그룹 데이터를 나타내는 `GroupBy` 클래스 객체를 반환한다. 이 객체에는 그룹별로 연산을 할 수 있는 그룹연산 메서드가 있다.

### 그룹연산 메서드

`groupby` 결과, 즉 `GroupBy` 클래스 객체의 뒤에 붙일 수 있는 그룹연산 메서드는 다양하다. 다음은 자주 사용되는 그룹연산 메서드들이다.

* `size`, `count`: 그룹 데이터의 갯수
* `mean`, `median`, `min`, `max`: 그룹 데이터의 평균, 중앙값, 최소, 최대
* `sum`, `prod`, `std`, `var`, `quantile` : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
* `first`, `last`: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터
   
이 외에도 많이 사용되는 것으로는 다음과 같은 그룹연산이 있다.

* `agg`, `aggregate`
    * 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 `agg`에 전달한다.
    * 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.

* `describe`
    * 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
   
* `apply`
    *  `describe` 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.
    
* `transform`
    * 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

예를 들어 다음과 같은 데이터가 있을 때 key1의 값(A 또는 B)에 따른 data1의 평균은 어떻게 구할까?

In [10]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


`groupby` 명령을 사용하여 그룹 A와 그룹 B로 구분한 그룹 데이터를 만든다.

In [12]:
groups = df2.groupby(df2.key1)
print(groups)
print(dir(groups))

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_accessors', '_agg_examples_doc', '_agg_general', '_agg_py_fallback', '_aggregate_frame', '_aggregate_item_by_item', '_aggregate_with_numba', '_apply_allowlist', '_apply_filter', '_apply_to_column_groupbys', '_bool_agg', '_can_use_transform_fast', '_choose_path', '_concat_objects', '_constructor', '_cumcount_array', '_cython_agg_general', '_cython_transform', '_define_paths', '_dir_additions', '_dir_deletions', '_fill', '_get_cythonized_result', '_get_data_to_aggregate', '_get_index', '_get_indices', '_goti

이 `GroupBy` 클래스 객체에는 각 그룹 데이터의 인덱스를 저장한 `groups` 속성이 있다.

In [13]:
print(groups.groups)

{'A': [0, 1, 4], 'B': [2, 3]}


A그룹과 B그룹 데이터의 합계를 구하기 위해 `sum`이라는 그룹연산을 한다.

In [14]:
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


`GroupBy` 클래스 객체를 명시적으로 얻을 필요가 없다면 `groupby` 메서드와 그룹연산 메서드를 연속으로 호출한다. 다음 예제는 열 `data1`에 대해서만 그룹연산을 하는 코드이다.

In [15]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

데이터를 그룹으로 나눈 `GroupBy` 클래스 객체 또는 그룹분석한 결과에서 `data1`만 뽑아도 된다.

In [16]:
df2.groupby(df2.key1)["data1"].sum()   # `GroupBy` 클래스 객체에서 data1만 선택하여 분석하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [17]:
df2.groupby(df2.key1).sum()["data1"]  # 전체 데이터를 분석한 후 data1만 선택한 경우

key1
A    8
B    7
Name: data1, dtype: int64

````{admonition} 연습 문제 4.7.1

key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다.
````

이번에는 복합 키 (key1, key2) 값에 따른 data1의 합계를 구하자. 
분석하고자 하는 키가 복수이면 리스트를 사용한다.

In [18]:
groups = df2.data1.groupby([df2.key1, df2.key2])
groups.groups

{('A', 'one'): [0, 4], ('A', 'two'): [1], ('B', 'one'): [2], ('B', 'two'): [3]}

In [19]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

이 결과를 `unstack` 명령으로 피봇 데이블 형태로 만들수도 있다.

In [20]:
df2.data1.groupby([df2["key1"], df2["key2"]]).groups

{('A', 'one'): [0, 4], ('A', 'two'): [1], ('B', 'one'): [2], ('B', 'two'): [3]}

In [21]:
df2.data1.groupby([df2["key1"], df2["key2"]]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [22]:
df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2")

key2,one,two
key1,,
A,6,2
B,3,4


그룹분석 기능을 사용하면 위의 인구 데이터로부터 지역별 합계를 구할 수도 있다.

In [ ]:
df1

In [ ]:
df1["인구"]

In [ ]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).groups

In [ ]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum()

In [ ]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")

다음 데이터는 150 송이의 붓꽃(iris)에 대해 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎너비(sepal_width), 꽃받침길이(petal_length), 꽃받침너비(petal_width)을 측정한 데이터이다. 

In [ ]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris

각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자. 이러한 계산을 하는 그룹연산 메서드는 없으므로 직접 만든 후 `agg` 메서드를 적용한다.

In [ ]:
def peak_to_peak_ratio(x):
#     print( type(x) )
#     print(x)
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

```{margin}
`describe`
```

`describe` 메서드를 사용하면 다양한 기술 통계(descriptive statistics)값을 한 번에 구한다. 그룹별로 하나의 스칼라 값이 아니라 하나의 데이터프레임이 생성된다는 점에 주의하라.

In [ ]:
iris.groupby(iris.species).describe()

In [ ]:
iris.groupby(iris.species).describe().T

```{margin}
`apply`
```

`apply` 메서드를 사용하면 `describe` 메서드처럼 하나의 그룹에 대해 하나의 대표값(스칼라 값)을 구하는 게 아니라 데이터프레임을 만들 수 있다. 예를 들어 다음처럼 각 붓꽃 종별로 가장 꽃잎 길이(petal length)가 큰 3개의 데이터를 뽑아낼 수도 있다.

In [ ]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

```{margin}
`transform`
```

`transform` 메서드는 그룹별 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터프레임 자체를 변화시킨다. 따라서 만들어진 데이터프레임의 크기는 원래 데이터프레임과 같다. 예를 들어 다음처럼 각 붓꽃 꽃잎길이가 해당 종 내에서  대/중/소 어느 것에 해당되는지에 대한 데이터프레임을 만들 수도 있다.

In [ ]:
def q3cut(s):
#     print("q3cut()")
#     print(s)
#     print(pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str))
    return pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str)


iris["petal_length_class"] = iris.groupby(iris.species).petal_length.transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)
iris

````{admonition} 연습 문제 4.7.2

붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라.
만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.
````

## `pivot_table`

```{margin}
`pivot_table`
```

Pandas는 `pivot` 명령과 `groupby` 명령의 중간 성격을 가지는 `pivot_table` 명령도 제공한다.

`pivot_table` 명령은 `groupby` 명령처럼 그룹분석을 하지만 최종적으로는 `pivot` 명령처럼 피봇테이블을 만든다. 즉 `groupby` 명령의 결과에 `unstack`을 자동 적용하여 2차원적인 형태로 변형한다. 사용 방법은 다음과 같다.

* `pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')`
    * `data`: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
    * `values`: 분석할 데이터프레임에서 분석할 열
    * `index`: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
    * `columns`: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
    * `aggfunc`: 분석 메서드
    * `fill_value`: NaN 대체 값
    * `margins`: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
    * `margins_name`: 마진 열(행)의 이름
    
만약 조건에 따른 데이터가 유일하게 선택되지 않으면 그룹연산을 하며 이 때 `aggfunc` 인수로 정의된 함수를 수행하여 대표값을 계산한다.

`pivot_table`를 메서드로 사용할 때는 객체 자체가 데이터가 되므로 `data` 인수가 필요하지 않다.

예를 들어 위에서 만들었던 피봇테이블은 `pivot_table` 명령으로 다음과 같이 만들 수도 있다. 인수의 순서에 주의하라.

In [ ]:
df1

In [ ]:
df1.pivot_table("인구", "도시", "연도")

`margins=True` 인수를 주면 `aggfunc`로 주어진 분석 방법을 해당 열의 모든 데이터, 해당 행의 모든 데이터 그리고 전체 데이터에 대해 적용한 결과를 같이 보여준다. `aggfunc`가 주어지지 않았으면 평균을 계산한다.

In [ ]:
df1.pivot_table("인구", "도시", "연도", margins=True, margins_name="평균")

이 결과에서 가장 오른쪽 합계 열의 첫번째 값 3451492은 모든 부산 인구 데이터의 평균, 두번째 값 9766113은 모든 서울 인구 데이터의 평균이다. 가장 아래의 합계 행의 첫번째 값은 2005년 데이터의 평균값, 두번째 값은 2010년 데이터의 평균값이다. 가장 오른쪽 아래의 값 5350809는 전체 데이터의 평균값이다. 다음 계산을 통해 이를 확인할 수 있다.

In [ ]:
df1["인구"].mean()

행 인덱스나 열 인덱스에 리스트를 넣으면 다중 인덱스 테이블을 만든다.

In [ ]:
df1.pivot_table("인구", index=["연도", "도시"])

식당에서 식사 후 내는 팁(tip)과 관련된 데이터를 이용하여 좀더 구체적으로 그룹분석 방법을 살펴본다. 우선 Seaborn 패키지에 설치된 샘플 데이터를 로드한다. 이 데이터프레임에서 각각의 컬럼은 다음을 뜻한다.

* total_bill: 식사대금
* tip: 팁
* sex: 성별
* smoker: 흡연/금연 여부
* day: 요일
* time: 시간
* size: 인원

In [ ]:
tips = sns.load_dataset("tips")
tips.tail()

분석의 목표는 식사 대금 대비 팁의 비율이 어떤 경우에 가장 높아지는지 찾는 것이다. 우선 식사대금와 팁의 비율을 나타내는 `tip_pct`를 추가하자.

In [ ]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

다음으로 각 열의 데이터에 대해 간단히 분포를 알아본다.

In [ ]:
tips.describe()

우선 성별로 나누어 데이터 갯수를 세어본다.

In [ ]:
tips.groupby("sex").count()

데이터 갯수의 경우 NaN 데이터가 없다면 모두 같은 값이 나올 것이다. 이 때는 `size` 명령을 사용하면 더 간단히 표시된다. `size` 명령은 NaN이 있어도 상관하지 않는다.

In [ ]:
tips.groupby("sex").size()

이번에는 성별과 흡연유무로 나누어 데이터의 갯수를 알아본다.

In [ ]:
tips.groupby(["sex", "smoker"]).size()

좀 더 보기 좋도록 피봇 데이블 형태로 바꿀 수도 있다.

In [ ]:
tips.pivot_table("tip_pct", "sex", "smoker", aggfunc="count", margins=True)

이제 성별과 흡연 여부에 따른 평균 팁 비율을 살펴본다.

In [ ]:
tips.groupby("sex")[["tip_pct"]].mean()

In [ ]:
tips.groupby("smoker")[["tip_pct"]].mean()

`pivot_table` 명령을 사용할 수도 있다.

In [ ]:
tips.pivot_table("tip_pct", "sex")

In [ ]:
tips.pivot_table("tip_pct", ["sex", "smoker"])

In [ ]:
tips.pivot_table("tip_pct", "sex", "smoker")

여성 혹은 흡연자의 팁 비율이 높은 것을 볼 수 있다. 하지만 이 데이터에는 평균을 제외한 분산(variance) 등의 다른 통계값이 없으므로 `describe` 명령으로 여러가지 통계값을 한 번에 알아본다.

In [ ]:
tips.groupby("sex")[["tip_pct"]].describe()

In [ ]:
tips.groupby("smoker")[["tip_pct"]].describe()

In [ ]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()

```{admonition} 연습 문제 4.7.3

1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.
2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?
````


이번에는 각 그룹에서 가장 많은 팁과 가장 적은 팁의 차이를 알아보자. 이 계산을 해 줄 수 있는 그룹연산 함수가 없으므로 함수를 직접 만들고 `agg` 메서드를 사용한다.

In [ ]:
def peak_to_peak(x):
    return x.max() - x.min()


tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

만약 여러가지 그룹연산을 동시에 하고 싶다면 다음과 같이 리스트를 이용한다.

In [ ]:
tips.groupby(["sex", "smoker"]).agg(["mean", peak_to_peak])[["total_bill"]]

만약 데이터 열마다 다른 연산을 하고 싶다면 열 라벨과 연산 이름(또는 함수)를 딕셔너리로 넣는다.

In [ ]:
tips.groupby(["sex", "smoker"]).agg(
    {'tip_pct': 'mean', 'total_bill': peak_to_peak})

다음은 `pivot_table` 명령으로 더 복잡한 분석을 한 예이다.

In [ ]:
tips.pivot_table(['tip_pct', 'size'], ['sex', 'day'], 'smoker')

In [ ]:
tips.pivot_table('size', ['time', 'sex', 'smoker'], 'day',
                 aggfunc='sum', fill_value=0)

````{admonition} 연습 문제 4.7.4

타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.

```
titanic = sns.load_dataset("titanic")
```

1. `qcut` 명령으로 세 개의 나이 그룹을 만든다.
2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.
생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.
3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.
````
